# `almop` Library Setup

I'm developping a python library for helping me encapsulate analysis of causal structures in time series data. 
The library setup is as follows:
1. Initialize the logger and specify the logging level
2. Import local `almop` package path so that python will be able to call all needed functions
3. Import `almop` modules and all other libraries

## Step 1 - Logging

In [ ]:
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(format='[%(levelname)s] %(asctime)s:\t %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Step 2 - System Path

In [ ]:
import sys
import os

if f"{os.getcwd()}/../" not in ' '.join(sys.path):
    logging.info('Local module \'almop\' not found in system path. Appending...')
    sys.path.append(f"{os.getcwd()}/../")

## Step 3 - Import Libraries

In [ ]:
#from almop import data as ald
from almop import data as ald
from almop import core as alc
from almop import gpu as alg
from almop import math as alm
import pandas as pd

## Parse Config

I'm using a configuration file that all notebooks can refer to. It contains useful parameters defining execution, such as the location on disk where the datasets should be persisted to.

**NOTE**: If you're running this for yourself, make sure to modify the config file appropriately. Everything else should then work smoothly.

In [ ]:
config = alc.parse_config('config.toml')
config

## Loading Data

In [ ]:
gdp = ald.get_gdp(config, ['PL', 'DE', 'IT', 'BE', 'SE'])

## Setting up `CuPy`

I will use the `gpu` part of the `almop` package to initialize interactions with the system's GPUs. The goal of the setup is to prepare the environment for executing CPU/GPU agnostic code.

In [ ]:
import cupy as cp 
import numpy as np

alg.set_memory_limits() # Set GPU V-RAM limits for all units

In [ ]:
a = np.array(gdp['PL'].dropna().values)

In [ ]:
cp.get_array_module(a)

In [ ]:
#a = cp.asarray(a)
gpu_result = alm.zscore(a)
#cpu_result = cp.asnumpy(gpu_result)
del gpu_result

In [ ]:
cpu_result